# Indexing, Selection, and Operations

Now that we are familiar with pandas' data structures, we can turn our attention to some of the intermediate features of data frames, which include:
    
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- **Hierarchical labeling** of axes
- **Sorting and ranking** of data in DataFrames
- Easy handling of **missing data**
- Data **summarization** tools

In this section, we will manipulate data collected from ocean-going vessels on the eastern seaboard. Vessel operations are monitored using the **Automatic Identification System (AIS)**, a safety at sea navigation technology which vessels are required to maintain and that uses transponders to transmit very high frequency (VHF) radio signals containing static information including ship name, call sign, and country of origin, as well as dynamic information unique to a particular voyage such as vessel location, heading, and speed.

![AIS](images/ais.gif)

The International Maritime Organization’s (IMO) International Convention for the Safety of Life at Sea requires functioning AIS capabilities on all vessels 300 gross tons or greater and the US Coast Guard requires AIS on nearly all vessels sailing in U.S. waters. The Coast Guard has established a national network of AIS receivers that provides coverage of nearly all U.S. waters. **AIS signals** are transmitted several times each minute and the network is capable of handling thousands of reports per minute and updates as often as every two seconds. Therefore, a typical voyage in our study might include the transmission of hundreds or thousands of AIS encoded signals. This provides a rich source of spatial data that includes both **spatial and temporal information**.

For our purposes, we will use **summarized data** that describes the transit of a given vessel through a particular administrative area. The data includes the start and end time of the transit segment, as well as information about the speed of the vessel, how far it travelled, etc.

In [3]:
import pandas as pd
import numpy as np

# vessels = pd.read_csv('sample_data/vessel_information.csv', index_col=0)

In [ ]:
vessels.head()

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
1,8,Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...,Y,Unknown,Unknown,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0,156.0,4,Dredging/MilOps/Reserved/Towing
9,3,000000009/Raven/Shearwater,N,Unknown,Unknown,2,50.0/62.0,62.0,2,Pleasure/Tug
21,1,Us Gov Vessel,Y,Unknown,Unknown,1,208.0,208.0,1,Unknown
74,2,Mcfaul/Sarah Bell,N,Unknown,Unknown,1,155.0,155.0,1,Unknown
103,3,Ron G/Us Navy Warship 103/Us Warship 103,Y,Unknown,Unknown,2,26.0/155.0,155.0,2,Tanker/Unknown


## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [ ]:
# Sample Series object
flag = vessels.flag
flag

mmsi
1            Unknown
9            Unknown
21           Unknown
74           Unknown
103          Unknown
              ...   
919191919    Unknown
967191190    Unknown
975318642    Unknown
987654321    Unknown
999999999    Unknown
Name: flag, Length: 10771, dtype: object

In [ ]:
# Numpy-style indexing
flag[:10]

mmsi
1                      Unknown
9                      Unknown
21                     Unknown
74                     Unknown
103                    Unknown
310                   Bermuda 
3011                 Anguilla 
4731      Yemen (Republic of) 
15151                  Unknown
46809    Syrian Arab Republic 
Name: flag, dtype: object

In [ ]:
# Indexing by label
flag[[298716,725011300]]

mmsi
298716       Unknown
725011300     Chile 
Name: flag, dtype: object

In a `DataFrame` we can slice along either or both axes:

In [ ]:
vessels[['num_names','num_types']].head()

,num_names,num_types
mmsi,,
1,8,4
9,3,2
21,1,1
74,2,1
103,3,2


In [ ]:
vessels[vessels.max_loa>700]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


The indexing field `loc` allows us to select subsets of rows and columns in an intuitive way:

In [ ]:
vessels.loc[720768000, ['names','flag', 'type']]

names             Rusty Pelican
flag     Bolivia (Republic of) 
type                     Towing
Name: 720768000, dtype: object

In [ ]:
vessels.loc[:4731, 'names']

mmsi
1       Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...
9                              000000009/Raven/Shearwater
21                                          Us Gov Vessel
74                                      Mcfaul/Sarah Bell
103              Ron G/Us Navy Warship 103/Us Warship 103
310                                              Arabella
3011                                           Charleston
4731                                            000004731
Name: names, dtype: object

Slicing also works with string variables, since an index has an intrinsic order, regardless of label:

In [ ]:
vessels.loc[:310, 'flag':'loa']

,flag,flag_type,num_loas,loa
mmsi,,,,
1,Unknown,Unknown,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0
9,Unknown,Unknown,2,50.0/62.0
21,Unknown,Unknown,1,208.0
74,Unknown,Unknown,1,155.0
103,Unknown,Unknown,2,26.0/155.0
310,Bermuda,Foreign,1,47.0


In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [ ]:
vessels.iloc[:5, 5:8]

,num_loas,loa,max_loa
mmsi,,,
1,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0,156.0
9,2,50.0/62.0,62.0
21,1,208.0,208.0
74,1,155.0,155.0
103,2,26.0/155.0,155.0


## Indexing with `where`

Pandas `DataFrame` objects also posess a `where` index for indexing that returns the values that satisfy the condition, but retain the index of the original `DataFrame`, so that the shape does not change. This is important when **alignment** is required for operations between `DataFrame`s.

In [ ]:
np.random.seed(42)
normal_vals = pd.DataFrame({'x{}'.format(i):np.random.randn(100) for i in range(5)})

normal_vals.head()

,x0,x1,x2,x3,x4
0,0.496714,-1.415371,0.357787,-0.828995,-1.594428
1,-0.138264,-0.420645,0.560785,-0.560181,-0.599375
2,0.647689,-0.342715,1.083051,0.747294,0.005244
3,1.523030,-0.802277,1.053802,0.610370,0.046981
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


In [ ]:
normal_vals.where(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,0.496714,NaN,0.357787,NaN,NaN
1,NaN,NaN,0.560785,NaN,NaN
2,0.647689,NaN,1.083051,0.747294,0.005244
3,1.523030,NaN,1.053802,0.610370,0.046981
4,NaN,NaN,NaN,NaN,NaN


`where` includes an optional `other` argument that accepts a scalar or tabular values (or a callable) to replace values in the `DataFrame` that do not satisfy the condition.

For example, we can use this to return the absolute values of `X`:

In [ ]:
normal_vals.where(normal_vals>0, other=-normal_vals).head()

,x0,x1,x2,x3,x4
0,0.496714,1.415371,0.357787,0.828995,1.594428
1,0.138264,0.420645,0.560785,0.560181,0.599375
2,0.647689,0.342715,1.083051,0.747294,0.005244
3,1.523030,0.802277,1.053802,0.610370,0.046981
4,0.234153,0.161286,1.377669,0.020902,0.450065


Similarly, a callable can be used when we need to modify the replaced value:

In [ ]:
normal_vals.where(normal_vals>0, other=lambda y: -y*100).head()

,x0,x1,x2,x3,x4
0,0.496714,141.537074,0.357787,82.899501,159.442766
1,13.826430,42.064532,0.560785,56.018104,59.937502
2,0.647689,34.271452,1.083051,0.747294,0.005244
3,1.523030,80.227727,1.053802,0.610370,0.046981
4,23.415337,16.128571,137.766937,2.090159,45.006547


Conversely, `mask` is the inverse boolean of `where`:

In [ ]:
normal_vals.mask(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,NaN,-1.415371,NaN,-0.828995,-1.594428
1,-0.138264,-0.420645,NaN,-0.560181,-0.599375
2,NaN,-0.342715,NaN,NaN,NaN
3,NaN,-0.802277,NaN,NaN,NaN
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


## Selection with `query`

At times, selection using indexing can be verbose because it requires repeated use of the `DataFrame` namespace.

In [ ]:
normal_vals[(normal_vals.x1 > normal_vals.x0) & (normal_vals.x3 > normal_vals.x2)].head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


For a more concise (and readable) syntax, we can use the new `query` method to perform selection on a `DataFrame`. Instead of having to type the fully-specified column, we can simply pass a string that describes what to select. The query above is then simply:

In [ ]:
normal_vals.query('(x1 > x0) & (x3 > x2)').head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


The `DataFrame.index` and `DataFrame.columns` are placed in the query namespace by default. If you want to refer to a variable in the current namespace, you can prefix the variable with `@`:

In [ ]:
min_loa = 700

In [ ]:
vessels.query('max_loa > @min_loa')

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For this section, we will import a complementary dataset of transits (voyages) of each vessel. This is a much larger dataset, since ships each have multiple transits over several years.

For example, we can perform arithmetic on the elements of two objects, such as combining distances travelled across years. First, let's artificially construct two Series, consisting of segment lengths travelled in 2009 and 2010:

In [4]:
segments = pd.read_csv('sample_data/transit_segments.csv', parse_dates=['st_time'])

Extract the year from the date (more on dates/times later!)

In [ ]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

0    2009.0
1    2009.0
2    2009.0
3    2009.0
4    2009.0
Name: year, dtype: float64

In [5]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum()
                         for name in segments2009.name.unique()})

KeyError: ignored

In [ ]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum()
                         for name in segments2010.name.unique()})

We will learn a much more efficient way of performing this operation when we get to `GroupBy` operations a little later on.

In [ ]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Now, let's add them together, in hopes of getting 2-year travel totals:

In [ ]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

+ Alida Gorthon +       NaN
+alida Gorthon+        63.0
+margit Gorthon+        NaN
205551000               NaN
210235000               NaN
                      ...  
Zim Shenzhen         1184.7
Zim Xiamen              NaN
Zuma                    NaN
\m/- /+                 NaN
stadt Gera              NaN
Length: 4309, dtype: float64

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, the majority of ships only have data for one of the two years.

In [ ]:
seg_lengths.notnull().mean()

0.3597122302158273

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate ship travel totals by using the `fill_value` argument to insert a zero for years that do not have data for a particular vessel:

In [ ]:
lengths2009.add(lengths2010, fill_value=0)

+ Alida Gorthon +     497.2
+alida Gorthon+        63.0
+margit Gorthon+      342.3
205551000               7.2
210235000               1.2
                      ...  
Zim Shenzhen         1184.7
Zim Xiamen             69.4
Zuma                   54.2
\m/- /+                17.2
stadt Gera             41.8
Length: 4309, dtype: float64

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum LOA (ship length) from the `max_loa` column, we get a the differences between the size of a given vessel to the largest vessel.

In [ ]:
vessels.max_loa - vessels.max_loa.max()

mmsi
1           -656.0
9           -750.0
21          -604.0
74          -657.0
103         -657.0
             ...  
919191919   -792.0
967191190   -781.0
975318642   -792.0
987654321   -789.0
999999999   -782.0
Name: max_loa, Length: 10771, dtype: float64

Or, looking at things row-wise, we can see how a particular vessel compares with the rest of the group with respect to important features:

In [ ]:
vessels[vessels.max_loa==vessels.max_loa.max()]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo


In [ ]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

,num_names,num_loas,max_loa,num_types
mmsi,,,,
1,7.0,4.0,-656.0,3.0
9,2.0,-1.0,-750.0,1.0
21,0.0,-2.0,-604.0,0.0
74,1.0,-2.0,-657.0,0.0
103,2.0,-1.0,-657.0,1.0
310,0.0,-2.0,-765.0,0.0
3011,0.0,-2.0,-652.0,0.0
4731,0.0,-2.0,-782.0,0.0
15151,1.0,-1.0,-637.0,0.0


We can also apply functions to each column or row of a `DataFrame`

In [ ]:
recs.apply(np.median)

num_names      1.0
num_loas       1.0
max_loa      183.0
num_types      1.0
dtype: float64

In [ ]:
def range_calc(x):
    return x.max() - x.min()

In [ ]:
recs.apply(range_calc)

num_names    167.0
num_loas      83.0
max_loa      812.0
num_types     13.0
dtype: float64

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [ ]:
segments.sort_index().head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,1,Us Govt Ves,1.0,1.0,5.1,13.2,9.2,14.5,96.5,2009-02-10 16:03:00,2/10/09 16:27,2009.0
1,1,Dredge Capt Frank,1.0,1.0,13.5,18.6,10.4,20.6,100.0,2009-04-06 14:31:00,4/6/09 15:20,2009.0
2,1,Us Gov Vessel,1.0,1.0,4.3,16.2,10.3,20.5,100.0,2009-04-06 14:36:00,4/6/09 14:55,2009.0
3,1,Us Gov Vessel,2.0,1.0,9.2,15.4,14.5,16.1,100.0,2009-04-10 17:58:00,4/10/09 18:34,2009.0
4,1,Dredge Capt Frank,2.0,1.0,9.2,15.4,14.6,16.2,100.0,2009-04-10 17:59:00,4/10/09 18:35,2009.0


In [ ]:
segments.sort_index(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
150983,367116570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
150982,367116570,Sorensen Miller,134.0,1.0,20.6,16.3,0.0,19.1,98.2,2010-07-20 08:51:00,7/20/10 10:08,2010.0
150981,367116570,Sorensen Miller,133.0,1.0,58.0,5.9,0.0,19.6,87.2,2010-07-19 13:14:00,7/20/10 3:49,2010.0
150980,367116570,Sorensen Miller,132.0,1.0,25.4,15.0,0.3,19.5,97.0,2010-07-18 00:15:00,7/18/10 1:55,2010.0
150979,367116570,Sorensen Miller,131.0,1.0,16.3,12.9,0.1,19.4,93.4,2010-07-17 13:53:00,7/17/10 15:27,2010.0


Try sorting the **columns** instead of the rows, in ascending order:

In [ ]:
segments.sort_index(axis=1).head()

,avg_sog,end_time,max_sog,min_sog,mmsi,name,pdgt10,seg_length,segment,st_time,transit,year
0,13.2,2/10/09 16:27,14.5,9.2,1,Us Govt Ves,96.5,5.1,1.0,2009-02-10 16:03:00,1.0,2009.0
1,18.6,4/6/09 15:20,20.6,10.4,1,Dredge Capt Frank,100.0,13.5,1.0,2009-04-06 14:31:00,1.0,2009.0
2,16.2,4/6/09 14:55,20.5,10.3,1,Us Gov Vessel,100.0,4.3,1.0,2009-04-06 14:36:00,1.0,2009.0
3,15.4,4/10/09 18:34,16.1,14.5,1,Us Gov Vessel,100.0,9.2,1.0,2009-04-10 17:58:00,2.0,2009.0
4,15.4,4/10/09 18:35,16.2,14.6,1,Dredge Capt Frank,100.0,9.2,1.0,2009-04-10 17:59:00,2.0,2009.0


We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [ ]:
segments.seg_length.sort_values(ascending=False).head(10)

149218    1675.3
49709     1647.8
149236    1644.2
50604     1557.0
106374    1517.3
140513    1502.4
140555    1491.6
50665     1489.2
140514    1442.9
140535    1413.1
Name: seg_length, dtype: float64

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [ ]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True],
                                           by=['max_sog', 'min_sog']).head(10)

,avg_sog,max_sog,min_sog
82222,1.5,49.9,0.0
119381,6.6,49.9,0.1
12479,12.2,49.9,3.5
119476,1.9,49.8,0.0
146139,9.6,49.8,0.1
91567,15.0,49.8,2.2
82041,10.5,49.8,7.0
93345,10.7,49.8,8.3
150262,7.1,49.3,4.8
78988,42.3,49.2,33.2


A related method `rank` does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [ ]:
segments.avg_sog.rank()

0         116862.5
1         146886.0
2         139367.5
3         134892.5
4         134892.5
            ...   
150979    113904.5
150980    132196.5
150981     16839.0
150982    139854.0
150983         NaN
Name: avg_sog, Length: 150984, dtype: float64

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [ ]:
pd.Series([100,100]).rank()

0    1.5
1    1.5
dtype: float64

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [ ]:
segments.avg_sog.rank(method='first').sort_values()

663            1.0
664            2.0
665            3.0
666            4.0
754            5.0
            ...   
73227     150980.0
73228     150981.0
78988     150982.0
137003    150983.0
150983         NaN
Name: avg_sog, Length: 150984, dtype: float64

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [ ]:
segments.rank(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,150980.5,6771.0,147612.0,82157.0,146032.5,34121.5,36553.0,52507.0,33440.0,140632.0,101374.0,125485.0
1,150980.5,112514.5,147612.0,82157.0,136385.5,4098.0,19296.0,7641.5,11811.5,135309.5,68673.0,125485.0
2,150980.5,6787.0,147612.0,82157.0,146677.5,11616.5,20217.0,7958.0,11811.5,135307.0,68674.0,125485.0
3,150980.5,6787.0,141099.5,82157.0,142092.0,16091.5,4741.0,34898.0,11811.5,134917.0,78758.0,125485.0
4,150980.5,112514.5,141099.5,82157.0,142092.0,16091.5,4592.5,34057.0,11811.5,134916.0,78757.0,125485.0


## Hierarchical indexing

While the `mmsi` (Maritime Mobile Service Identity) field represents a unique identifier in the `vessels` table, it does not in the `segments` table.

We can use hierarchical indexing to build a **unique index** that may be more meaningful than the ordered set of integers that are given as a default index.

In [ ]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head(10)

seg_length  avg_sog  min_sog  \
mmsi name              year   transit segment                                 
1    Bil Holman Dredge 2010.0 1.0     1.0            17.4     34.3     33.9   
                                      2.0            76.0     34.4     33.8   
     Dredge Capt Frank 2009.0 1.0     1.0            13.5     18.6     10.4   
                              2.0     1.0             9.2     15.4     14.6   
     S.d. Gumel        2011.0 1.0     1.0            13.7      6.9      3.0   
     Us Gov Vessel     2009.0 1.0     1.0             4.3     16.2     10.3   
                              2.0     1.0             9.2     15.4     14.5   
     Us Govt Ves       2009.0 1.0     1.0             5.1     13.2      9.2   
9    Shearwater        2010.0 4.0     1.0            11.6      8.8      7.9   
                              8.0     1.0            11.6      8.3      7.4   

                                               max_sog  pdgt10  \
mmsi name              year   transit segment                    
1    Bil Holman Dredge 2010.0 1.0     1.0         35.0   100.0   
                                      2.0         34.9   100.0   
     Dredge Capt Frank 2009.0 1.0     1.0         20.6   100.0   
                              2.0     1.0         16.2   100.0   
     S.d. Gumel        2011.0 1.0     1.0         14.0    38.2   
     Us Gov Vessel     2009.0 1.0     1.0         20.5   100.0   
                              2.0     1.0         16.1   100.0   
     Us Govt Ves       2009.0 1.0     1.0         14.5    96.5   
9    Shearwater        2010.0 4.0     1.0         10.3     0.0   
                              8.0     1.0          9.2     0.0   

                                                          st_time  \
mmsi name              year   transit segment                       
1    Bil Holman Dredge 2010.0 1.0     1.0     2010-03-20 16:06:00   
                                      2.0     2010-03-20 18:05:00   
     Dredge Capt Frank 2009.0 1.0     1.0     2009-04-06 14:31:00   
                              2.0     1.0     2009-04-10 17:59:00   
     S.d. Gumel        2011.0 1.0     1.0     2011-05-04 11:28:00   
     Us Gov Vessel     2009.0 1.0     1.0     2009-04-06 14:36:00   
                              2.0     1.0     2009-04-10 17:58:00   
     Us Govt Ves       2009.0 1.0     1.0     2009-02-10 16:03:00   
9    Shearwater        2010.0 4.0     1.0     2010-06-05 11:23:00   
                              8.0     1.0     2010-06-08 11:03:00   

                                                    end_time  
mmsi name              year   transit segment                 
1    Bil Holman Dredge 2010.0 1.0     1.0      3/20/10 16:36  
                                      2.0      3/20/10 18:48  
     Dredge Capt Frank 2009.0 1.0     1.0       4/6/09 15:20  
                              2.0     1.0      4/10/09 18:35  
     S.d. Gumel        2011.0 1.0     1.0       5/4/11 13:30  
     Us Gov Vessel     2009.0 1.0     1.0       4/6/09 14:55  
                              2.0     1.0      4/10/09 18:34  
     Us Govt Ves       2009.0 1.0     1.0      2/10/09 16:27  
9    Shearwater        2010.0 4.0     1.0       6/5/10 12:44  
                              8.0     1.0       6/8/10 12:29

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [ ]:
segments_hier.index.is_unique

True

Try using this hierarchical index to retrieve the first segment of the tenth transit of the *Sentinel* (mmsi=366766980):

In [ ]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

seg_length                   45.5
avg_sog                       9.8
min_sog                       6.0
max_sog                      10.4
pdgt10                       37.5
st_time       2009-01-23 07:08:00
end_time            1/23/09 10:49
Name: (366766980, Sentinel, 2009.0, 10.0, 1.0), dtype: object

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [ ]:
segments.loc[9]

mmsi                            9
name                   Shearwater
transit                       8.0
segment                       1.0
seg_length                   11.6
avg_sog                       8.3
min_sog                       7.4
max_sog                       9.2
pdgt10                        0.0
st_time       2010-06-08 11:03:00
end_time             6/8/10 12:29
year                       2010.0
Name: 9, dtype: object

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)),
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

If you want to get fancy, both the row and column indices themselves can be given names:

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With this, we can do all sorts of custom indexing:

In [ ]:
frame.loc['a', 'Ohio']

,Green,Red
1,0,1
2,3,4


Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [ ]:
segments_hier.swaplevel('mmsi', 'name').head()

seg_length  avg_sog  min_sog  \
name              mmsi year   transit segment                                 
Bil Holman Dredge 1    2010.0 1.0     1.0            17.4     34.3     33.9   
                                      2.0            76.0     34.4     33.8   
Dredge Capt Frank 1    2009.0 1.0     1.0            13.5     18.6     10.4   
                              2.0     1.0             9.2     15.4     14.6   
S.d. Gumel        1    2011.0 1.0     1.0            13.7      6.9      3.0   

                                               max_sog  pdgt10  \
name              mmsi year   transit segment                    
Bil Holman Dredge 1    2010.0 1.0     1.0         35.0   100.0   
                                      2.0         34.9   100.0   
Dredge Capt Frank 1    2009.0 1.0     1.0         20.6   100.0   
                              2.0     1.0         16.2   100.0   
S.d. Gumel        1    2011.0 1.0     1.0         14.0    38.2   

                                                          st_time  \
name              mmsi year   transit segment                       
Bil Holman Dredge 1    2010.0 1.0     1.0     2010-03-20 16:06:00   
                                      2.0     2010-03-20 18:05:00   
Dredge Capt Frank 1    2009.0 1.0     1.0     2009-04-06 14:31:00   
                              2.0     1.0     2009-04-10 17:59:00   
S.d. Gumel        1    2011.0 1.0     1.0     2011-05-04 11:28:00   

                                                    end_time  
name              mmsi year   transit segment                 
Bil Holman Dredge 1    2010.0 1.0     1.0      3/20/10 16:36  
                                      2.0      3/20/10 18:48  
Dredge Capt Frank 1    2009.0 1.0     1.0       4/6/09 15:20  
                              2.0     1.0      4/10/09 18:35  
S.d. Gumel        1    2011.0 1.0     1.0       5/4/11 13:30

In [ ]:
segments_hier.swaplevel('mmsi', 'name').loc['Sentinel'].head()

seg_length  avg_sog  min_sog  max_sog  \
mmsi      year   transit segment                                          
366766980 2008.0 1.0     1.0            44.6      6.6      4.9      9.3   
                 2.0     1.0            33.6      8.2      4.6      8.9   
                 3.0     1.0            27.0      6.7      2.8     11.3   
                 4.0     1.0            35.4     10.0      6.7     10.4   
                 5.0     1.0           100.9      5.2      2.6      8.9   

                                  pdgt10             st_time        end_time  
mmsi      year   transit segment                                              
366766980 2008.0 1.0     1.0         0.0 2008-11-10 05:00:00  11/10/08 11:49  
                 2.0     1.0         0.0 2008-11-12 09:53:00  11/12/08 14:00  
                 3.0     1.0        10.0 2008-11-24 07:02:00  11/24/08 11:01  
                 4.0     1.0        69.1 2008-12-05 20:20:00   12/5/08 23:53  
                 5.0     1.0         0.0 2008-12-17 00:52:00  12/17/08 20:18

Alternately, one can index based on an arbitrary level using the `xs` (cross-section) method:

In [ ]:
segments_hier.xs('Sentinel', level='name').head()

seg_length  avg_sog  min_sog  max_sog  \
mmsi      year   transit segment                                          
366766980 2008.0 1.0     1.0            44.6      6.6      4.9      9.3   
                 2.0     1.0            33.6      8.2      4.6      8.9   
                 3.0     1.0            27.0      6.7      2.8     11.3   
                 4.0     1.0            35.4     10.0      6.7     10.4   
                 5.0     1.0           100.9      5.2      2.6      8.9   

                                  pdgt10             st_time        end_time  
mmsi      year   transit segment                                              
366766980 2008.0 1.0     1.0         0.0 2008-11-10 05:00:00  11/10/08 11:49  
                 2.0     1.0         0.0 2008-11-12 09:53:00  11/12/08 14:00  
                 3.0     1.0        10.0 2008-11-24 07:02:00  11/24/08 11:01  
                 4.0     1.0        69.1 2008-12-05 20:20:00   12/5/08 23:53  
                 5.0     1.0         0.0 2008-12-17 00:52:00  12/17/08 20:18

Perhaps the easiest way to extract data from arbitrary levels of a `MultiIndex` is to use the `IndexSlice` function. For example, if we want the first two transits from all trips in 2009 and 2010:

In [ ]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

mmsi  name               year  transit  segment
1     Bil Holman Dredge  2010  1        1          17.4
                                        2          76.0
      Dredge Capt Frank  2009  1        1          13.5
                               2        1           9.2
      Us Gov Vessel      2009  1        1           4.3
                               2        1           9.2
      Us Govt Ves        2009  1        1           5.1
74    Mcfaul             2009  1        2           1.4
      Sarah Bell         2009  1        1           7.4
103   Ron G              2009  1        1          87.5
                                        3           1.4
                               2        1          30.7
Name: seg_length, dtype: float64

The `:` tells pandas to include the entire level. What `IndexSlice` does is generate a Python `slice` object that represents what is required from each level. So, the above is equivalent to:

    (slice(None, None, None), slice(None, None, None), [2009, 2010], [1, 2])

Data can also be sorted by any index level, using `sort_index` with the `level` argument appropriately specified:

In [ ]:
segments.sort_index(level='max_sog', ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
262525,999999999,Triple Attraction,8,1,19.8,18.6,16.1,19.5,100.0,2010-06-18 10:19:00,6/18/10 11:22,2010
262524,999999999,Triple Attraction,7,1,31.5,14.2,13.4,15.1,100.0,2010-06-18 02:52:00,6/18/10 5:03,2010
262523,999999999,Triple Attraction,6,1,17.4,17.0,14.7,18.4,100.0,2010-06-17 19:16:00,6/17/10 20:17,2010
262522,999999999,Triple Attraction,4,1,18.7,19.2,18.4,19.9,100.0,2010-06-15 21:32:00,6/15/10 22:29,2010
262521,999999999,Triple Attraction,3,1,5.3,20.0,19.6,20.4,100.0,2010-06-15 12:49:00,6/15/10 13:05,2010


## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

0       NaN
1        -3
2      None
3    foobar
dtype: object

In [ ]:
foo.isnull()

0     True
1    False
2     True
3    False
dtype: bool

To illustrate how to deal with missing values in pandas, we will import an educational database, which reports test scores for children with hearing disabilities, along with associated covariates, several of which are missing.

In [ ]:
testing = pd.read_csv('sample_data/test_scores.csv', index_col=0)
testing.head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,NaN,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


Missing values may be dropped or indexed out:

By default, `dropna` drops entire rows in which one or more values are missing.

In [ ]:
testing.dropna().head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False
14,91,0,2.0,0.0,False,0.0,58,1.0,1.0,True,True
19,65,1,0.0,2.0,False,0.0,50,1.0,1.0,False,True
21,71,1,1.0,2.0,False,1.0,58,0.0,0.0,True,True
23,82,0,1.0,0.0,False,1.0,59,0.0,1.0,True,False
25,97,0,1.0,0.0,False,1.0,56,1.0,1.0,False,False
26,95,1,1.0,0.0,True,0.0,55,0.0,1.0,True,False
27,98,1,1.0,0.0,False,0.0,52,0.0,1.0,True,False
32,115,0,1.0,0.0,False,0.0,55,0.0,1.0,False,False


A similar result can be obtained via indexing.

In [ ]:
testing.isnull().head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,False,False,False,False,False,True,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,True,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,True,False,False


In [ ]:
# testing.notnull()
testing[testing.notnull()].head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False


This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [ ]:
testing.dropna(how='all')

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
220,68,0,2.0,2.0,False,0.0,53,0.0,1.0,True,False
221,104,1,2.0,0.0,False,0.0,52,1.0,1.0,False,False
222,71,1,1.0,NaN,False,0.0,57,1.0,NaN,False,True
223,118,0,2.0,0.0,False,0.0,57,1.0,NaN,True,False


This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [ ]:
testing.dropna(thresh=10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
219,80,0,1.0,NaN,False,1.0,52,1.0,1.0,False,False
220,68,0,2.0,2.0,False,0.0,53,0.0,1.0,True,False
221,104,1,2.0,0.0,False,0.0,52,1.0,1.0,False,False
223,118,0,2.0,0.0,False,0.0,57,1.0,NaN,True,False


This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [ ]:
# Write your answer here
testing.dropna(axis = 1)
# testing.head(10)

,score,male,siblings,non_english,age_test,non_severe_hl,early_ident,non_white
0,40,0,2.0,False,55,1.0,False,False
1,31,1,0.0,False,53,0.0,False,False
2,83,1,1.0,True,52,1.0,False,True
3,75,0,3.0,False,55,0.0,False,False
5,62,0,0.0,False,50,0.0,False,False
...,...,...,...,...,...,...,...,...
220,68,0,2.0,False,53,0.0,True,False
221,104,1,2.0,False,52,1.0,False,False
222,71,1,1.0,False,57,1.0,False,True
223,118,0,2.0,False,57,1.0,True,False


Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [ ]:
testing.fillna(0).head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,0.0,55,1.0,0.0,False,False
1,31,1,0.0,0.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,0.0,False,True
3,75,0,3.0,0.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,0.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,0.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,0.0,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,0.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


In [ ]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,1.0,False,False
1,31,1,0.0,0.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,1.0,False,True
3,75,0,3.0,0.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,1.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,1.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

Missing values can also be interpolated, using any one of a variety of methods:

In [ ]:
testing.fillna(method='bfill').head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,0.0,55,1.0,0.0,False,False
1,31,1,0.0,1.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,1.0,False,True
3,75,0,3.0,4.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,1.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,1.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,0.0,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [ ]:
testing.sum()

score            17428.0
male               106.0
siblings           233.0
family_inv         170.0
non_english         42.0
prev_disab          45.0
age_test         11084.0
non_severe_hl       86.0
mother_hs           94.0
early_ident         78.0
non_white          100.0
dtype: float64

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [ ]:
testing.mean()

score            84.193237
male              0.512077
siblings          1.125604
family_inv        0.977011
non_english       0.202899
prev_disab        0.238095
age_test         53.545894
non_severe_hl     0.415459
mother_hs         0.701493
early_ident       0.376812
non_white         0.483092
dtype: float64

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
testing.mean(skipna=False)

score            84.193237
male              0.512077
siblings          1.125604
family_inv             NaN
non_english       0.202899
prev_disab             NaN
age_test         53.545894
non_severe_hl     0.415459
mother_hs              NaN
early_ident       0.376812
non_white         0.483092
dtype: float64

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

0      0.0
1      0.0
2      1.0
3      0.0
5      0.0
      ... 
220    0.0
221    0.0
222    0.0
223    0.0
224    0.0
Length: 207, dtype: float64

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [ ]:
testing.describe()

,score,male,siblings,family_inv,prev_disab,age_test,non_severe_hl,mother_hs
count,207.000000,207.000000,207.000000,174.000000,189.000000,207.000000,207.000000,134.000000
mean,84.193237,0.512077,1.125604,0.977011,0.238095,53.545894,0.415459,0.701493
std,24.885590,0.501066,0.866682,1.002621,0.427049,3.631920,0.493996,0.459320
min,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000
25%,68.000000,0.000000,1.000000,0.000000,0.000000,50.000000,0.000000,0.000000
50%,87.000000,1.000000,1.000000,1.000000,0.000000,54.000000,0.000000,1.000000
75%,98.000000,1.000000,2.000000,2.000000,0.000000,57.000000,1.000000,1.000000
max,144.000000,1.000000,3.000000,4.000000,1.000000,59.000000,1.000000,1.000000


`describe` can detect non-numeric data and sometimes yield useful information about it.

In [ ]:
testing.non_english.describe()

count       207
unique        2
top       False
freq        165
Name: non_english, dtype: object

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
testing.score.cov(testing.age_test)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
testing.score.corr(testing.age_test)

Try running `corr` on the entire `testing` DataFrame to see what is returned:

In [ ]:
# Write answer here

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
segments_hier.head()

In [ ]:
segments_hier.sum(level='transit').seg_length

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [ ]:
testing.to_csv("testing.csv")

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

An efficient way of storing data to disk is in binary format. Pandas supports this using Python’s built-in pickle serialization.

In [ ]:
testing.to_pickle("testing_pickle")

The complement to `to_pickle` is the `read_pickle` function, which restores the pickle to a `DataFrame` or `Series`:

In [ ]:
pd.read_pickle("testing_pickle").head()

As Wes warns in his book, it is recommended that binary storage of data via pickle only be used as a temporary storage format, in situations where speed is relevant. This is because there is no guarantee that the pickle format will not change with future versions of Python.

## References

[Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney